In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    #setting the derivatives
    #for our ODE
    
    
    y_derivs = np.zeros(2)
    #dydx = z
    y_derivs[0] = y[1]
    #dzdx=-7
    y_derivs[1] = -1*y[0]
    return y_derivs

In [ ]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #1/2 step
    x_ipoh = xi + 0.5*h
    #x at 1 step
    x_ipo = xi + h
    
    #temp array
    y_temp = np.zeros(nv)
    #k1 val
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    #k2 val
    y_temp [:]  = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #k3 val
    y_temp [:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    #k4 val
    y_temp [:] = yi[:] + k3[:]
    y_derivds = dydx(x_ipoh,y_temp)
    k4[:] = h*y_derivs[:]
    
    #step y up
    yipo = yi + (k1+ 2*k2 + 2*k3 + k4)/6
                  
    return yipo

In [ ]:

def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #max num of iterations and starting var for iter
    imax = 10000
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #step
    h_step = h
    #step adjust
    while (Delta.max()/tol > 1.0):
        
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #error
        Delta = np.fabs(y_2 - y_11)
        #make step smaller if too big
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        i+=1
    
    #take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    
    return y_2, h_new, h_step

In [ ]:
def rk4_mv(dfdx,a,b,y_a,tol):
    #dfdx-derivative
    #a-lower bound
    #b-upper bound
    #y_a-bound conditions
    #toler-tolerance for y intergration
    
    #starting steps
    xi = a
    yi = y_a.copy()
    
    #initial step
    h = 1.0e-4 * (b-a)
    
    #iterations(inital and max)
    imax = 10000
    i = 0
    
    #num of coupled odes
    nv = len(y_a)
    
    #initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    
    flag = 1
    
    while(flag):
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            h = b-xi
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            flag = 0 #break
        
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
    
        if(i>imax):
            print("Maximum iteratuibs reached.")
            raise StopIteration("Iteration number = ", i)
        i+=1
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        if(xi==b):
            flag = 0
            
    return x,y

## main

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0

nv = 2

tolerance = 1.0e-6

x,y = rk4_mv(dydx,a,b,y_0,tolerance)

In [ ]:
plt.plot(x,y[:,0], 'o',label='y(x)')
plt.plot(x,y[:,1], 'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)
plt.show()

In [ ]:
sine= np.sin(x)
cosine=np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dy/dx(x) Error")
plt.legend(frameon=False)
plt.show()